In [ ]:
!unzip -o "/content/drive/MyDrive/t5-basemodel.zip" -d "/content/my_model_folder"

Archive:  /content/drive/MyDrive/t5-basemodel.zip
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/model.safetensors  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/scheduler.pt  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/spiece.model  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/rng_state.pth  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/special_tokens_map.json  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/generation_config.json  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/config.json  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/optimizer.pt  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/tokenizer.json  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/tokenizer_config.json  
  inflating: /content/my_model_folder/20251027_03_checkpoint-36900/trainer_state.json  
  inf

In [ ]:
#必要なライブラリをインストール
!pip install fastapi uvicorn pyngrok nest-asyncio transformers sentencepiece torch
!pip install sentencepiece

In [ ]:
import nest_asyncio
from pyngrok import ngrok
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from google.colab import drive
import threading #並列処理

# ngrokの認証
NGROK_AUTHTOKEN = '34dpPaKymmAHuTmypudiLTaNLny_7hdXik5Bb2hHuawQbGKng'
os.environ['NGROK_AUTHTOKEN'] = NGROK_AUTHTOKEN

try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Driveマウント済み、またはエラー: {e}")

# モデルをロード
MODEL_PATH = "/content/my_model_folder/20251027_03_checkpoint-36900"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = None
tokenizer = None

try:
    print(f"モデルをロード中です (パス: {MODEL_PATH})...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)
    model.to(device)
    print(f"✅ モデルのロードに成功しました！ ({device}を使用)")

except Exception as e:
    print(f"🚨エラー: モデルのロードに失敗。パスの確認。")

# FastAPIアプリを初期化
app = FastAPI()#土台

# CORSの設定 (重要)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# リクエストのデータ形式を定義(サーバに送られてくるJSONは "text": 文文章章"の形式)
class RequestText(BaseModel):
    text: str

# APIエンドポイントの定義-窓口
@app.post("/predict") #/predictというurlへのPOSTリクエストが来たらrun_predictionを実行
async def run_prediction(request: RequestText):

    if model is None or tokenizer is None:
        return {"error": "サーバー側でモデルがロードされていません"}

    input_text = request.text
    print(f"受け取ったテキスト: {input_text}")

    # T5モデルを使って予測（生成）を実行
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=512)
    result_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print(f"返却するひらがな: {result_text}")

    # 結果をJSONとして返す
    results = {
        "input": input_text,
        "prediction": result_text
    }

    return results


# uvicornをバックグラウンドで実行するための関数
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000) #サーバを起動する関数

# ngrokで公開URLを取得
try:
    public_url = ngrok.connect(8000) #トンネル作成, 公開urlの作成
    print(f"✅ サーバーが起動しました。公開URL: {public_url}")
    print(f"Chrome拡張機能には {public_url}/predict を設定してください。")
except Exception as e:
    print(f"🚨 ngrokの起動に失敗しました。")
    print(f"エラー詳細: {e}")
    print("セッションを再起動して、もう一度お試しください。")

# サーバーを別スレッド(バックグラウンド)で起動-ランタイムを再起動で停止。
thread = threading.Thread(target=run_server)
thread.start()

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Driveマウント済み、またはエラー: mount failed
モデルをロード中です (パス: /content/my_model_folder/20251027_03_checkpoint-36900)...
✅ モデルのロードに成功しました！ (cudaを使用)
✅ サーバーが起動しました。公開URL: NgrokTunnel: "https://piacular-semisocialistically-aviana.ngrok-free.dev" -> "http://localhost:8000"
Chrome拡張機能には NgrokTunnel: "https://piacular-semisocialistically-aviana.ngrok-free.dev" -> "http://localhost:8000"/predict を設定してください。
